# Ayudantía 14: Extracción y visualización de datos #
diego.herrerag00@uc.cl

## Importar librerias

In [13]:
import requests
import pandas as pd
import os
from getpass import getpass
from IPython.display import display
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

In [14]:
os.environ["API_KEY"] = getpass("Ingresa tu API KEY de OpenWeatherMap: ")
API_KEY = os.getenv("API_KEY")

## Ejercicio Formativo 1 Capítulo 7

### Misión 1

Crear funcion para obtener información de la API.

In [15]:
def get_weather(ciudad, api_key):
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    url = f"https://api.openweathermap.org/data/2.5/weather?q={ciudad}&appid={api_key}&units=metric&lang=es"
    try:
        response = requests.get(url, params=params)
        data = response.json()

        if response.status_code != 200:
            print(f"Error al consultar {ciudad}: {data.get('message')}")
            return None

        weather_info = {
            "Ciudad": ciudad,
            "Temperatura": data["main"]["temp"],
            "Sensación": data["main"]["feels_like"],
            "Clima": data["weather"][0]["description"],
            "Humedad": data["main"]["humidity"],
            "Viento": data["wind"]["speed"]
        }
        return weather_info

    except Exception as e:
        print(f"Error al obtener datos para {ciudad}: {e}")
        return None

Función para crear DataFrame de la información obtenida.

In [16]:
def get_weather_multi(ciudades, api_key):
    datos = []
    for ciudad in ciudades:
        info = get_weather(ciudad, api_key)
        if info:
            datos.append(info)
    return pd.DataFrame(datos)

Mostrar resultados.

In [17]:
ciudades = ["Santiago", "Buenos Aires", "Lima", "Madrid", "Ciudad de México"]
df_clima = get_weather_multi(ciudades, API_KEY)
df_clima

,Ciudad,Temperatura,Sensación,Clima,Humedad,Viento
0,Santiago,30.24,29.52,cielo claro,36,2.68
1,Buenos Aires,24.43,24.31,cielo claro,53,4.63
2,Lima,20.14,20.12,nubes dispersas,73,5.14
3,Madrid,11.30,10.91,muy nuboso,93,8.75
4,Ciudad de México,22.75,21.60,cielo claro,20,3.09


### Misión 2

Cargar los datasets `df_countries` y `df_cities`.

In [18]:
df_countries = pd.read_csv("countries.csv")
df_cities = pd.read_csv("cities.csv")
print("Paises:")
display(df_countries.head())
print("Ciudades:")
display(df_cities.head())

Paises:


,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
0,Afghanistan,افغانستان,AF,AFG,26023100.0,652230.0,Kabul,34.526011,69.177684,Southern and Central Asia,Asia
1,Albania,Shqipëria,AL,ALB,2895947.0,28748.0,Tirana,41.326873,19.818791,Southern Europe,Europe
2,Algeria,الجزائر,DZ,DZA,38700000.0,2381741.0,Algiers,36.775361,3.060188,Northern Africa,Africa
3,American Samoa,American Samoa,AS,ASM,55519.0,199.0,Pago Pago,-14.275479,-170.704830,Polynesia,Oceania
4,Angola,Angola,AO,AGO,24383301.0,1246700.0,Luanda,-8.827270,13.243951,Central Africa,Africa


Ciudades:


,station_id,city_name,country,state,iso2,iso3,latitude,longitude
0,41515,Asadabad,Afghanistan,Kunar,AF,AFG,34.866000,71.150005
1,38954,Fayzabad,Afghanistan,Badakhshan,AF,AFG,37.129761,70.579247
2,41560,Jalalabad,Afghanistan,Nangarhar,AF,AFG,34.441527,70.436103
3,38947,Kunduz,Afghanistan,Kunduz,AF,AFG,36.727951,68.872530
4,38987,Qala i Naw,Afghanistan,Badghis,AF,AFG,34.983000,63.133300


Seleccionar al menos 10 ciudades de 5 países distintos.

Ciudades de Sudamérica:

In [19]:
df_countries[df_countries['continent'] == "South America"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
179,Suriname,Suriname,SR,SUR,534189.0,163820.0,Paramaribo,5.821609,-55.177043,South America,South America
145,Paraguay,Paraguay,PY,PRY,6893727.0,406752.0,Asunción,-25.280046,-57.634381,South America,South America
7,Argentina,Argentina,AR,ARG,42669500.0,2780400.0,Buenos Aires,-34.607568,-58.437089,South America,South America
53,Ecuador,Ecuador,EC,ECU,15888900.0,276841.0,Quito,-0.220164,-78.512327,South America,South America
79,Guyana,Guyana,GY,GUY,784894.0,214969.0,Georgetown,6.802577,-58.162861,South America,South America


Ciudades de Africa:

In [20]:
df_countries[df_countries['continent'] == "Africa"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
118,Mali,Mali,ML,MLI,15768000.0,1240192.0,Bamako,12.605033,-7.986514,Western Africa,Africa
163,Senegal,Sénégal,SN,SEN,13508715.0,196722.0,Dakar,14.693425,-17.447938,Western Africa,Africa
210,Western Sahara,الصحراء الغربية,EH,ESH,586000.0,266000.0,El Aaiún,27.154512,-13.195392,Northern Africa,Africa
155,Rwanda,Rwanda,RW,RWA,10996891.0,26338.0,Kigali,-1.885960,30.129675,Eastern Africa,Africa
172,South Africa,South Africa,ZA,ZAF,54002000.0,1221037.0,Pretoria,-25.745937,28.187944,Southern Africa,Africa


Ciudades de Asia:

In [21]:
df_countries[df_countries['continent'] == "Asia"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
14,Bangladesh,বাংলাদেশ,BD,BGD,157486000.0,147570.0,Dhaka,23.759357,90.378814,Southern and Central Asia,Asia
20,Bhutan,ʼbrug-yul,BT,BTN,755030.0,38394.0,Thimphu,27.472762,89.629548,Southern and Central Asia,Asia
207,Vietnam,Việt Nam,VN,VNM,89708900.0,331212.0,Hanoi,21.029450,105.854444,Southeast Asia,Asia
195,Turkey,Türkiye,TR,TUR,76667864.0,783562.0,Ankara,39.920777,32.854067,Middle East,Asia
196,Turkmenistan,Türkmenistan,TM,TKM,5838064.0,488100.0,Ashgabat,37.939668,58.387426,Southern and Central Asia,Asia


Ciudades de Europa:

In [22]:
df_countries[df_countries['continent'] == "Europe"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
169,Slovenia,Slovenija,SI,SVN,2064966.0,20273.0,Ljubljana,46.049980,14.506860,Southern Europe,Europe
125,Moldova,Moldova,MD,MDA,3557600.0,33846.0,Chișinău,47.024471,28.832253,Eastern Europe,Europe
119,Malta,Malta,MT,MLT,416055.0,316.0,Valletta,35.898982,14.513676,Southern Europe,Europe
22,Bosnia and Herzegovina,Bosna i Hercegovina,BA,BIH,3791622.0,51209.0,Sarajevo,43.851977,18.386687,Southern Europe,Europe
153,Romania,România,RO,ROU,19942642.0,238391.0,Bucharest,44.436141,26.102720,Eastern Europe,Europe


Ciudades de Oceania:

In [23]:
df_countries[df_countries['continent'] == "Oceania"].sample(5)

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
138,Norfolk Island,Norfolk Island,NF,NFK,2302.0,36.0,Kingston,17.971215,-76.792813,Australia and New Zealand,Oceania
160,Samoa,Samoa,WS,WSM,187820.0,2842.0,Apia,-13.834369,-171.769279,Polynesia,Oceania
74,Guam,Guam,GU,GUM,159358.0,549.0,Hagåtña,13.472745,144.752018,Micronesia,Oceania
3,American Samoa,American Samoa,AS,ASM,55519.0,199.0,Pago Pago,-14.275479,-170.704830,Polynesia,Oceania
205,Vanuatu,Vanuatu,VU,VUT,264652.0,12189.0,Port Vila,-17.741497,168.315016,Melanesia,Oceania


In [24]:
ciudades_seleccionadas = [
    ("Santiago", "Chile"),
    ("Buenos Aires", "Argentina"),
    ("Nairobi", "South Africa"),
    ("Madrid", "Kenya"),
    ("Doha", "Qatar"),
    ("Tokyo", "Japón"),
    ("London", "United Kingdom"),
    ("Berlin", "Germany"),
    ("Canberra", "Australia"),
    ("Suva", "Fiji")
]

Obtener el clima actual.

In [25]:
nombres_ciudades = [c[0] for c in ciudades_seleccionadas]
df_clima = get_weather_multi(nombres_ciudades, API_KEY)

Añadir columna de país y calcular la diferencia térmica.

In [26]:
df_clima["País"] = [c[1] for c in ciudades_seleccionadas]
df_clima["Diferencia"] = abs(df_clima["Temperatura"] - df_clima["Sensación"])

Reordenar las columnas y ordenar el DataFrame.

In [27]:
columnas_ordenadas = [
    "Ciudad", "País", "Temperatura", "Sensación", "Diferencia",
    "Clima", "Humedad", "Viento"
]

df_clima = df_clima[columnas_ordenadas].sort_values(by="Diferencia", ascending=False)

Mostrar los resultados.

In [28]:
df_clima

,Ciudad,País,Temperatura,Sensación,Diferencia,Clima,Humedad,Viento
7,Berlin,Germany,1.95,-1.60,3.55,muy nuboso,88,3.58
6,London,United Kingdom,6.75,3.25,3.50,muy nuboso,83,5.66
5,Tokyo,Japón,11.18,10.13,1.05,muy nuboso,68,4.12
9,Suva,Fiji,23.71,24.75,1.04,muy nuboso,100,1.54
0,Santiago,Chile,30.24,29.52,0.72,cielo claro,36,2.68
3,Madrid,Kenya,11.30,10.91,0.39,muy nuboso,93,8.75
4,Doha,Qatar,22.97,23.31,0.34,cielo claro,76,3.60
2,Nairobi,South Africa,18.93,18.76,0.17,muy nuboso,72,5.14
8,Canberra,Australia,14.04,13.88,0.16,nubes,91,1.54
1,Buenos Aires,Argentina,24.43,24.31,0.12,cielo claro,53,4.63


Observamos que la mayor diferencia entre temperatura real y sensación térmica se da en **Canberra, Australia**. Esto podría deberse a factores como el viento o la humedad relativa. Ciudades con climas secos tienden a tener menor diferencia. También notamos que varias ciudades tienen sensación más baja que la temperatura real, lo cual es común en ambientes ventosos.

### Misión 3

Renombrar columnas y cruzar datos.

In [29]:
df_countries_renombrada = df_countries.rename(columns={"country": "País"})
df_clima_merge = df_clima.merge(
    df_countries_renombrada[["País", "population", "area", "continent", "region"]],
    on="País",
    how="left"
)
df_clima_merge

,Ciudad,País,Temperatura,Sensación,Diferencia,Clima,Humedad,Viento,population,area,continent,region
0,Berlin,Germany,1.95,-1.60,3.55,muy nuboso,88,3.58,80783000.0,357114.0,Europe,Western Europe
1,London,United Kingdom,6.75,3.25,3.50,muy nuboso,83,5.66,64105654.0,242900.0,Europe,British Isles
2,Tokyo,Japón,11.18,10.13,1.05,muy nuboso,68,4.12,NaN,NaN,NaN,NaN
3,Suva,Fiji,23.71,24.75,1.04,muy nuboso,100,1.54,859178.0,18272.0,Oceania,Melanesia
4,Santiago,Chile,30.24,29.52,0.72,cielo claro,36,2.68,17819054.0,756102.0,South America,South America
5,Madrid,Kenya,11.30,10.91,0.39,muy nuboso,93,8.75,41800000.0,580367.0,Africa,Eastern Africa
6,Doha,Qatar,22.97,23.31,0.34,cielo claro,76,3.60,2269672.0,11586.0,Asia,Middle East
7,Nairobi,South Africa,18.93,18.76,0.17,muy nuboso,72,5.14,54002000.0,1221037.0,Africa,Southern Africa
8,Canberra,Australia,14.04,13.88,0.16,nubes,91,1.54,NaN,7692024.0,Oceania,Australia and New Zealand
9,Buenos Aires,Argentina,24.43,24.31,0.12,cielo claro,53,4.63,42669500.0,2780400.0,South America,South America


Agrupar por continente y analizar.

In [30]:
resumen_continente = df_clima_merge.groupby("continent").agg({
    "Temperatura": "mean",
    "Sensación": "mean",
    "Diferencia": "mean",
    "Humedad": "mean",
    "Viento": "mean"
}).round(2).reset_index()

resumen_continente

,continent,Temperatura,Sensación,Diferencia,Humedad,Viento
0,Africa,15.12,14.84,0.28,82.5,6.94
1,Asia,22.97,23.31,0.34,76.0,3.60
2,Europe,4.35,0.82,3.52,85.5,4.62
3,Oceania,18.88,19.32,0.60,95.5,1.54
4,South America,27.34,26.92,0.42,44.5,3.66


Al agrupar por continente, se observa que las condiciones climáticas varían bastante entre regiones. Por ejemplo, **Europa** muestra temperaturas más bajas en promedio, mientras que **Asia** presenta menor humedad en comparación a **Europa** y **Oceanía**. Las diferencias térmicas mayores se ven en regiones con climas extremos como **Europa**. Esta comparación ayuda a entender patrones globales y posibles influencias geográficas.

### Misión 4

Importar la tabla desde Wikipedia.

In [ ]:
url = "https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_meteorol%C3%B3gicos_mundiales"

headers = {
    "User-Agent": "Mozilla/5.0"  # simula un navegador
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()   # Esta linea lo que hace es lanzar error si no es 200, para lanzar una excepción y detener el programa en ese punto. 

tablas = pd.read_html(resp.text)

for i, tabla in enumerate(tablas):
    print(f"Tabla {i}: {tabla.columns}")
    display(tabla.head())


Guardamos tabla de temperaturas.

In [ ]:
df_records = tablas[0].copy()
df_records.columns = df_records.columns.get_level_values(0)
df_records.head()

,Unnamed: 0_level_0,Temperatura,Ubicación,Fecha
0,Estados Unidos,"56,7 °C (134 °F)",Valle de la Muerte (California),1913-07-10[8]​
1,México,"52,5 °C (126,3 °F)",San Luis Río Colorado (Sonora),1966-06-15[9]​
2,Canadá,"49,6 °C (121,3 °F)",Lytton (Columbia Británica),2021-06-29[10]​
3,Groenlandia,"30,1 °C (86,2 °F)",Ivittuut,1915-06-23[11]​
4,América Central y Antillas[editar],América Central y Antillas[editar],América Central y Antillas[editar],América Central y Antillas[editar]


Renombrar columnas y ajustar temperatura a float.

In [ ]:
df_records.columns = ["País", "Temperatura", "Lugar", "Fecha"]
df_records["Temperatura"] = (
    df_records["Temperatura"]
    .astype(str)
    .str.extract(r"(\d+,\d+|\d+\.\d+|\d+)")
    .replace(",", ".", regex=True)
    .astype(float)
)
df_records.head()

,País,Temperatura,Lugar,Fecha
0,Estados Unidos,56.7,Valle de la Muerte (California),1913-07-10[8]​
1,México,52.5,San Luis Río Colorado (Sonora),1966-06-15[9]​
2,Canadá,49.6,Lytton (Columbia Británica),2021-06-29[10]​
3,Groenlandia,30.1,Ivittuut,1915-06-23[11]​
4,América Central y Antillas[editar],NaN,América Central y Antillas[editar],América Central y Antillas[editar]


Renombrar columnas y cruzar datos.

In [ ]:
df_records_merge = df_records[["País", "Temperatura"]].rename(columns={"Temperatura": "Máxima histórica"})
df_clima_ext = df_clima_merge.merge(df_records_merge, on="País", how="left")

Verificar valores faltantes.

In [ ]:
faltantes = df_clima_ext[df_clima_ext["Máxima histórica"].isna()]["País"].unique()
print("Países sin datos históricos:", faltantes)

Países sin datos históricos: ['Germany' 'United Kingdom' 'Kenya' 'Fiji' 'South Africa' 'Qatar']


Comparar temperatura actual con la histórica.

In [ ]:
df_clima_ext["Excede histórica (%)"] = (
    (df_clima_ext["Temperatura"] / df_clima_ext["Máxima histórica"]) * 100
).round(2)
df_clima_ext

,Ciudad,País,Temperatura,Sensación,Diferencia,Clima,Humedad,Viento,population,area,continent,region,Máxima histórica,Excede histórica (%)
0,Berlin,Germany,2.39,-2.92,5.31,muy nuboso,90,7.20,80783000.0,357114.0,Europe,Western Europe,NaN,NaN
1,London,United Kingdom,3.37,0.48,2.89,muy nuboso,77,3.09,64105654.0,242900.0,Europe,British Isles,NaN,NaN
2,Tokyo,Japón,8.41,6.55,1.86,cielo claro,62,3.09,NaN,NaN,NaN,NaN,41.8,20.12
3,Madrid,Kenya,7.66,6.00,1.66,cielo claro,53,2.57,41800000.0,580367.0,Africa,Eastern Africa,NaN,NaN
4,Suva,Fiji,21.71,22.55,0.84,algo de nubes,100,0.00,859178.0,18272.0,Oceania,Melanesia,NaN,NaN
5,Canberra,Australia,11.54,10.73,0.81,nubes,76,4.12,NaN,7692024.0,Oceania,Australia and New Zealand,50.7,22.76
6,Nairobi,South Africa,19.93,19.55,0.38,algo de nubes,60,5.66,54002000.0,1221037.0,Africa,Southern Africa,NaN,NaN
7,Doha,Qatar,25.57,25.78,0.21,cielo claro,61,4.63,2269672.0,11586.0,Asia,Middle East,NaN,NaN
8,Santiago,Chile,22.12,22.03,0.09,nubes,63,0.45,17819054.0,756102.0,South America,South America,43.0,51.44
9,Buenos Aires,Argentina,21.02,21.11,0.09,nubes,74,4.12,42669500.0,2780400.0,South America,South America,49.1,42.81


 En esta misión realizamos web scraping de una tabla de Wikipedia con temperaturas máximas históricas por país, y comparamos estos valores con las temperaturas actuales de distintas ciudades. Observamos que la mayoría de las ciudades tienen temperaturas actuales significativamente por debajo del récord histórico de su país. Esto es esperable, ya que los récords suelen ocurrir en condiciones extremas y localizadas, como desiertos o durante olas de calor inusuales.

Algunas dificultades encontradas fueron las diferencias en nombres de países entre fuentes, y la localización específica de las mediciones históricas, que muchas veces no coinciden con las ciudades analizadas.

## Ejercicio Formativo 2 Capítulo 7

### Misión 1

Estructura base de la app.

In [ ]:
# Inicializamos la app con Dash y Bootstrap
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Ponemos nuestra API Key de OpenWeatherMap

# Función para obtener el clima actual
def get_weather(ciudad, api_key):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if response.status_code != 200:
            return f"Error: {data.get('message')}"

        weather_info = {
            "Ciudad": ciudad,
            "Temperatura": f"{data['main']['temp']} °C",
            "Sensación térmica": f"{data['main']['feels_like']} °C",
            "Clima": data['weather'][0]['description'],
            "Humedad": f"{data['main']['humidity']}%",
            "Viento": f"{data['wind']['speed']} m/s"
        }
        return weather_info
    except Exception as e:
        return f"Error: {e}"

Layout de la app.

In [ ]:
app.layout = dbc.Container([
    html.H2("Clima actual por ciudad", className="my-3"),

    dbc.Row([
        dbc.Col([
            dcc.Input(id="input-ciudad", type="text", placeholder="Ingresa una ciudad", className="form-control"),
        ], width=6),

        dbc.Col([
            dbc.Button("Consultar", id="btn-clima", color="primary", className="me-2")
        ], width="auto")
    ], className="mb-4"),

    html.Div(id="resultado-clima")
])

Callback para actualizar la tarjeta con el clima.


In [ ]:
@app.callback(
    Output("resultado-clima", "children"),
    Input("btn-clima", "n_clicks"),
    State("input-ciudad", "value")
)
def actualizar_clima(n_clicks, ciudad):
    if not ciudad:
        return dbc.Alert("Por favor ingresa una ciudad.", color="warning")

    info = get_weather(ciudad, API_KEY)

    if isinstance(info, str):
        return dbc.Alert(info, color="danger")

    return dbc.Card([
        dbc.CardHeader(html.H5(f"Clima en {info['Ciudad']}")),
        dbc.CardBody([
            html.P(f"Temperatura: {info['Temperatura']}"),
            html.P(f"Sensación térmica: {info['Sensación térmica']}"),
            html.P(f"Clima: {info['Clima']}"),
            html.P(f"Humedad: {info['Humedad']}"),
            html.P(f"Viento: {info['Viento']}")
        ])
    ], color="light", className="mb-3")

 Ejecutar la aplicación.

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

### Misión 2

Modificamos el layout.

In [ ]:
# Ponemos nuestra API Key de OpenWeatherMap
API_KEY = "CAMBIAR"

# Listamos las ciudades sugeridas.
ciudades_disponibles = [
    "Santiago", "Buenos Aires", "Lima", "Madrid", "Ciudad de México",
    "Paris", "Tokyo", "Cairo", "New York", "Sydney"
]

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Función para obtener información del clima
def get_weather(ciudad, api_key):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    try:
        r = requests.get(url, params=params)
        data = r.json()
        if r.status_code != 200:
            return None

        return {
            "Ciudad": ciudad,
            "Temperatura": data["main"]["temp"],
            "Sensación térmica": data["main"]["feels_like"],
            "Clima": data["weather"][0]["description"],
            "Humedad": data["main"]["humidity"],
            "Viento": data["wind"]["speed"]
        }
    except:
        return None

app.layout = dbc.Container([
    html.H2("Comparación de clima en múltiples ciudades", className="my-3"),

    dcc.Dropdown(
        id="dropdown-ciudades",
        options=[{"label": c, "value": c} for c in ciudades_disponibles],
        multi=True,
        placeholder="Selecciona ciudades"
    ),

    dbc.Button("Consultar clima", id="btn-multi", color="primary", className="mt-3"),

    html.Div(id="tarjetas-clima", className="mt-4"),
    dcc.Graph(id="grafico-temperaturas", className="mt-4")
])

modo_visualizacion = "historica" 

Crear callback para tarjetas y gráfico.

In [ ]:
def crear_figura_vacia(titulo):
    return px.bar(
        pd.DataFrame({"Ciudad": [], "Temperatura": []}),
        x="Ciudad",
        y="Temperatura",
        title=titulo
    )

@app.callback(
    [Output("tarjetas-clima", "children"),
     Output("grafico-temperaturas", "figure")],
    Input("btn-multi", "n_clicks"),
    Input("dropdown-ciudades", "value")
)
def actualizar_ciudades(n_clicks, ciudades):
    titulo_basico = "Temperatura actual por ciudad"
    titulo_historico = "Temperatura actual por ciudad vs. récord histórico"
    historicos = globals().get("dict_hist_temp", {})

    if not isinstance(historicos, dict):
        historicos = {}

    if not ciudades:
        mensaje = dbc.Alert("Selecciona al menos una ciudad.", color="warning")
        figura = (crear_figura_vacia(titulo_historico)
                  if modo_visualizacion == "historica"
                  else crear_figura_vacia(titulo_basico))
        return mensaje, figura

    datos = []
    tarjetas = []

    for ciudad in ciudades:
        info = get_weather(ciudad, API_KEY)
        if not info:
            continue

        if modo_visualizacion == "historica":
            nombre_pais = info.get("País", "?")
            max_hist = None
            mapa_codigos = {"CL": "Chile", "AR": "Argentina", "MX": "México",
                            "ES": "España", "FR": "Francia", "JP": "Japón",
                            "US": "Estados Unidos", "AU": "Australia",
                            "EG": "Egipto", "KE": "Kenia"}
            pais_nombre = mapa_codigos.get(nombre_pais, nombre_pais)

            if pais_nombre in historicos:
                max_hist = historicos[pais_nombre]
                porcentaje = round((info["Temperatura"] / max_hist) * 100, 1)
            else:
                porcentaje = None

            datos.append({**info, "Máxima histórica": max_hist, "% de récord": porcentaje,
                          "Pais legible": pais_nombre})

            color_tarjeta = "danger" if porcentaje and porcentaje > 90 else "light"
            tarjetas.append(
                dbc.Card([
                    dbc.CardHeader(html.H5(f"{info['Ciudad']} ({pais_nombre})")),
                    dbc.CardBody([
                        html.P(f"Temperatura: {info['Temperatura']} °C"),
                        html.P(f"Sensación: {info['Sensación térmica']} °C"),
                        html.P(f"Clima: {info['Clima']}"),
                        html.P(f"Humedad: {info['Humedad']}%"),
                        html.P(f"Viento: {info['Viento']} m/s"),
                        html.Hr(),
                        html.P(f"Máxima histórica país: {max_hist} °C" if max_hist else "❔ No hay dato histórico"),
                        html.P(f"% del récord: {porcentaje}%" if porcentaje else "")
                    ])
                ], color=color_tarjeta, className="mb-3")
            )
        else:
            datos.append(info)
            tarjetas.append(
                dbc.Card([
                    dbc.CardHeader(html.H5(info['Ciudad'])),
                    dbc.CardBody([
                        html.P(f"Temperatura: {info['Temperatura']} °C"),
                        html.P(f"Sensación: {info['Sensación térmica']} °C"),
                        html.P(f"Clima: {info['Clima']}"),
                        html.P(f"Humedad: {info['Humedad']}%"),
                        html.P(f"Viento: {info['Viento']} m/s"),
                    ])
                ], color="light", className="mb-3")
            )

    if not datos:
        mensaje = dbc.Alert(
            "No pudimos obtener datos. Revisa tu API Key o intenta con otras ciudades.",
            color="danger"
        )
        figura = (crear_figura_vacia(titulo_historico)
                  if modo_visualizacion == "historica"
                  else crear_figura_vacia(titulo_basico))
        return mensaje, figura

    df = pd.DataFrame(datos)

    if modo_visualizacion == "historica":
        fig = px.bar(
            df,
            x="Ciudad",
            y="Temperatura",
            color="% de récord",
            title=titulo_historico
        )
    else:
        fig = px.bar(
            df,
            x="Ciudad",
            y="Temperatura",
            color="Ciudad",
            title=titulo_basico
        )

    return tarjetas, fig

 Ejecutar la app.

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

### Misión 3

Cargar los datos históricos de temperatura.

In [ ]:
# Web scraping desde Wikipedia
url = "https://es.wikipedia.org/wiki/Anexo:R%C3%A9cords_meteorol%C3%B3gicos_mundiales"

# Descargamos el HTML con cabecera de navegador, ya que la página de Wikipedia bloquea el scraping, y al agregar la cabecera, se puede acceder a la página.
headers = {"User-Agent": "Mozilla/5.0"}
resp = requests.get(url, headers=headers)
resp.raise_for_status()          # lanza error si no es 200

# Leemos las tablas a partir del HTML descargado y las almacenamos en una variable.
tablas = pd.read_html(resp.text)

# Aca lo que hacemos es copiar la primera tabla y la almacenamos en un DataFrame.
df_records = tablas[0].copy()
df_records.columns = df_records.columns.get_level_values(1)
df_records.columns = ["País", "Temperatura", "Lugar", "Fecha"]

# Limpiamos la columna de temperatura
df_records["Temperatura"] = (
    df_records["Temperatura"]
    .astype(str)
    .str.extract(r"(\d+,\d+|\d+\.\d+|\d+)")
    .replace(",", ".", regex=True)
    .astype(float)
)

# Diccionario de temperaturas históricas por país
dict_hist_temp = df_records.set_index("País")["Temperatura"].to_dict()


/var/folders/z5/wvf1lf5579ld90s3v_k6rdt00000gn/T/ipykernel_10769/4106719321.py:10: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Agregar país a cada ciudad y obtener máxima histórica.

In [ ]:
def get_weather(ciudad, api_key):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": ciudad,
        "appid": api_key,
        "units": "metric",
        "lang": "es"
    }
    try:
        r = requests.get(url, params=params)
        data = r.json()
        if r.status_code != 200:
            return None

        # Nombre del país 
        codigo_pais = data["sys"]["country"]

        return {
            "Ciudad": ciudad,
            "País": codigo_pais,
            "Temperatura": data["main"]["temp"],
            "Sensación térmica": data["main"]["feels_like"],
            "Clima": data["weather"][0]["description"],
            "Humedad": data["main"]["humidity"],
            "Viento": data["wind"]["speed"]
        }
    except:
        return None